In [1]:
pip install nltk scikit-learn googlesearch-python wikipedia-api beautifulsoup4 requests paramiko


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json
import random
import sympy as sp
import re
import datetime
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# File to store chat history
CHAT_HISTORY_FILE = "chat_history.json"
MODEL_FILE = "chatbot_model.pkl"

# Load training data from JSON file
with open("bot.json", "r") as file:
    data = json.load(file)

responses = data["responses"]
medical_questions = data["medical_questions"]
jokes = data["jokes"]
fun_facts = data["fun_facts"]
medical_facts = data["medical_facts"]

# Merge all question categories
all_responses = {**responses, **medical_questions}

# Prepare corpus and responses
corpus = list(all_responses.keys())
response_values = list(all_responses.values())

def train_and_save_model():
    """Trains the model and saves it to a file."""
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(corpus).toarray()
    
    with open(MODEL_FILE, "wb") as model_file:
        pickle.dump((vectorizer, vectors, response_values), model_file)
    print("Model trained and saved!")

def load_model():
    """Loads the trained model if it exists."""
    try:
        with open(MODEL_FILE, "rb") as model_file:
            return pickle.load(model_file)
    except FileNotFoundError:
        return None

def load_chat_history():
    """Loads chat history if the file exists, otherwise creates an empty history."""
    try:
        with open(CHAT_HISTORY_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def save_chat_history(chat_history):
    """Saves chat history to the file."""
    with open(CHAT_HISTORY_FILE, "w") as file:
        json.dump(chat_history, file, indent=4)

# Load or train model
model_data = load_model()
if model_data:
    vectorizer, vectors, response_values = model_data
else:
    train_and_save_model()
    model_data = load_model()
    vectorizer, vectors, response_values = model_data

chat_history = load_chat_history()

def detect_and_solve_math(user_input):
    user_input_cleaned = re.sub(r"(what is|calculate|do|find|solve|compute|work out|answer)", "", user_input, flags=re.IGNORECASE).strip()
    user_input_cleaned = user_input_cleaned.replace("x", "*")
    
    if re.search(r"[\+\-\*/\^]", user_input_cleaned):
        try:
            result = sp.sympify(user_input_cleaned)
            return f"The answer is: {result}"
        except:
            return "Sorry, I couldn't solve that math problem."
    return None

def get_time():
    return "The current time is: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def chatbot_response(user_input):
    math_response = detect_and_solve_math(user_input)
    if math_response:
        return math_response

    user_vector = vectorizer.transform([user_input]).toarray()
    similarities = cosine_similarity(user_vector, vectors)
    best_match_index = np.argmax(similarities)

    if similarities[0][best_match_index] == 0:
        return "I'm sorry, I don't understand that.(The Model is still under training)"

    response = response_values[best_match_index]

    if response == "get_joke":
        return random.choice(jokes)
    elif response == "get_fact":
        return random.choice(fun_facts)
    elif response == "get_medical_fact":
        return random.choice(medical_facts)
    elif response == "get_time":
        return get_time()
    
    return response

print("ChatBot: Hello! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    print(f"user: {user_input}")
    user_input = user_input.lower()
    if user_input == "quit":
        print("ChatBot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"ChatBot: {response}")
    chat_history.append({"user": user_input, "bot": response})
    save_chat_history(chat_history)

In [2]:
import json
import random
import sympy as sp
import re
import datetime
import pickle
import numpy as np
import os
import subprocess
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

VENV_PYTHON1 = r"c:\Users\super\Desktop\DESKTOP\Final Year project\PROJECT MEDIC\.venv\Scripts\python.exe"
# Load training data from JSON file
if os.path.exists("bot.json"):
    with open("bot.json", "r") as file:
        data = json.load(file)
else:
    data = {"responses": {}, "medical_questions": {}, "jokes": [], "fun_facts": [], "medical_facts": []}

responses = data.get("responses", {})
medical_questions = data.get("medical_questions", {})
jokes = data.get("jokes", [])
fun_facts = data.get("fun_facts", [])
medical_facts = data.get("medical_facts", [])

# Merge all question categories
all_responses = {**responses, **medical_questions}

# Prepare corpus and responses
corpus = list(all_responses.keys())  # User inputs
response_values = list(all_responses.values())  # Bot responses

# Chat history file
CHAT_HISTORY_FILE = "chat_history.json"

def save_chat(user_input, bot_response):
    """Saves chat history to a JSON file."""
    history = []
    if os.path.exists(CHAT_HISTORY_FILE):
        with open(CHAT_HISTORY_FILE, "r") as file:
            history = json.load(file)
    
    history.append({"user": user_input, "bot": bot_response})
    
    with open(CHAT_HISTORY_FILE, "w") as file:
        json.dump(history, file, indent=4)

# Check if model exists
MODEL_FILE = "chatbot_model.pkl"

def train_and_save_model():
    """Trains the model and saves it to a file."""
    vectorizer = CountVectorizer()
    vectors = vectorizer.fit_transform(corpus).toarray()
    
    with open(MODEL_FILE, "wb") as model_file:
        pickle.dump((vectorizer, vectors, response_values), model_file)

def load_model():
    """Loads the trained model if it exists."""
    try:
        with open(MODEL_FILE, "rb") as model_file:
            return pickle.load(model_file)
    except FileNotFoundError:
        return None

# Load or train model
model_data = load_model()
if model_data:
    vectorizer, vectors, response_values = model_data
else:
    train_and_save_model()
    model_data = load_model()
    vectorizer, vectors, response_values = model_data

# Function to detect and solve math expressions
def detect_and_solve_math(user_input):
    user_input_cleaned = re.sub(r"(what is|calculate|do|find|solve|compute|work out|answer)", "", user_input, flags=re.IGNORECASE).strip()
    user_input_cleaned = user_input_cleaned.replace("x", "*")
    
    if re.search(r"[\+\-\*/\^]", user_input_cleaned):
        try:
            result = sp.sympify(user_input_cleaned)
            return f"The answer is: {result}"
        except (sp.SympifyError, TypeError):
            return "Sorry, I couldn't solve that math problem."
    
    return None

# Function to get current system time
def get_time():
    return "The current time is: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Function to check health parameters using Raspberry Pi
HEALTH_CHECK_SCRIPT = "health_check.py"

def check_health():
    """Runs health_check.py within the virtual environment and returns JSON output."""

    # Prompt user to select a body part for measurement
    print("Select body part for temperature measurement:")
    print("1: Forehead\n2: Wrist\n3: Armpit\n4: Chest\n5: Back")
    
    selected_option = input("Enter your choice (1-5): ").strip()
    
    # Validate input
    if selected_option not in ["1", "2", "3", "4", "5"]:
        return "Invalid selection. Please enter a number between 1 and 5."

    try:
        # Run health_check.py inside the virtual environment with the selected option
        process = subprocess.Popen(
            [VENV_PYTHON1, HEALTH_CHECK_SCRIPT, selected_option],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
        )
        stdout, stderr = process.communicate()

        if stderr:
            return f"Error checking health: {stderr.strip()}"

        # Ensure clean JSON parsing
        try:
            data = json.loads(stdout.strip())
        except json.JSONDecodeError:
            return "Error: Invalid JSON format received from health_check.py"

        if "error" in data:
            return f"Error: {data['error']}"

        # Display the formatted result
        return (f"📊 Final Body Temperature Estimate:\n"
                f"🌡️ Average {data['body_part']} Temperature: {data['avg_body_temp_C']}°C ({data['avg_body_temp_F']}°F)\n"
                "✅ Measurement completed successfully!")

    except Exception as e:
        return f"Error checking health: {str(e)}"


# Function to get chatbot response
def chatbot_response(user_input):
    user_input = user_input.strip().lower()  # Normalize input

    # Check if the input is a math problem
    math_response = detect_and_solve_math(user_input)
    if math_response:
        save_chat(user_input, math_response)
        return math_response
    
    # Check for health-related queries
    health_keywords = ["check my health", "i have", "i feel", "fever", "heart rate", "temperature"]
    if any(keyword in user_input for keyword in health_keywords):
        health_response = check_health()
        save_chat(user_input, health_response)
        return health_response
    
    # Process text-based queries
    user_vector = vectorizer.transform([user_input]).toarray()
    similarities = cosine_similarity(user_vector, vectors)
    best_match_index = np.argmax(similarities)
    
    if similarities[0][best_match_index] == 0:
        bot_response = "I'm sorry, I don't understand that."
    else:
        bot_response = response_values[best_match_index]
    
    # Check for special responses
    special_responses = {
        "get_joke": random.choice(jokes),
        "get_fact": random.choice(fun_facts),
        "get_medical_fact": random.choice(medical_facts),
        "get_time": get_time()
    }
    bot_response = special_responses.get(bot_response, bot_response)

    save_chat(user_input, bot_response)
    return bot_response

# Run chatbot
if __name__ == "__main__":
    print("ChatBot: Hello! Type 'Quit' to exit.")
    while True:
        user_input = input("You: ").strip()
        print(f"User: {user_input}")
        if user_input.lower() == "quit":
            print("ChatBot: Goodbye!")
            break
        print(f"ChatBot: {chatbot_response(user_input)}")


ChatBot: Hello! Type 'Quit' to exit.
User: what is anemia
ChatBot: Anemia is a condition where the blood lacks enough healthy red blood cells to carry oxygen.
User: hii
ChatBot: Hello! How can I assist you today?
User: quit
ChatBot: Goodbye!


In [4]:
import json
import paramiko
import time

# Raspberry Pi connection details
RASPBERRY_PI_HOSTNAME = "raspberrypi.local"  # Use IP if needed
USERNAME = "pi"
PASSWORD = "raspberry"  # Change if necessary
REMOTE_SCRIPT = "/home/pi/data.py"
VENV_PATH = "/home/pi/fyp/tf_venv/bin/activate"  # Virtual environment path

def send_command_to_raspberry(action, selected_option=None):
    """Send a command to Raspberry Pi and retrieve sensor data."""
    try:
        client = paramiko.SSHClient()
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        client.connect(RASPBERRY_PI_HOSTNAME, username=USERNAME, password=PASSWORD)

        # Build command based on action
        if action == "temperature":
            command = f"source {VENV_PATH} && python3 {REMOTE_SCRIPT} temperature {selected_option}"
        elif action == "max30102":
            command = f"source {VENV_PATH} && python3 {REMOTE_SCRIPT} max30102"
        else:
            return {"error": "Invalid action"}

        stdin, stdout, stderr = client.exec_command(command)

        output = stdout.read().decode().strip()
        errors = stderr.read().decode().strip()

        client.close()

        if errors:
            return {"error": errors}

        # Convert output to JSON format
        try:
            return json.loads(output)
        except json.JSONDecodeError:
            return {"error": "Invalid JSON format received from Raspberry Pi"}

    except Exception as e:
        return {"error": str(e)}

def analyze_health(temp, heart_rate, spo2):
    """Analyze health based on temperature, heart rate, and SpO₂."""
    health_status = []
    suggestions = []

    # Check temperature
    if temp < 36.0:
        health_status.append("⚠️ Low Body Temperature (Hypothermia)")
        suggestions.append("🧣 Wear warm clothing and stay in a heated place.")
    elif temp > 37.5:
        health_status.append("⚠️ Fever detected")
        suggestions.append("💧 Drink fluids, rest, and take fever-reducing medication if needed.")
    else:
        health_status.append("✅ Normal Temperature")

    # Check heart rate
    if heart_rate < 60:
        health_status.append("⚠️ Low Heart Rate (Bradycardia)")
        suggestions.append("🏃 Engage in light exercise to boost heart rate.")
    elif heart_rate > 100:
        health_status.append("⚠️ High Heart Rate (Tachycardia)")
        suggestions.append("🧘 Try relaxation techniques (deep breathing, meditation).")
    else:
        health_status.append("✅ Normal Heart Rate")

    # Check SpO₂
    if spo2 < 90:
        health_status.append("⚠️ Severe Hypoxia (Low Oxygen Level)")
        suggestions.append("🚑 Seek immediate medical attention!")
    elif spo2 < 95:
        health_status.append("⚠️ Mild Hypoxia (Monitor closely)")
        suggestions.append("🌬️ Take deep breaths and get fresh air.")
    else:
        health_status.append("✅ Normal Oxygen Levels")

    return health_status, suggestions

if __name__ == "__main__":
    selected_option = input("Enter body part option:\n 1: Forehead\n 2: Wrist\n 3: Armpit\n 4: Chest\n 5: Back \nYour Option: ").strip()
    print("Measuring temperature, please wait...")

    # Step 1: Get temperature from Raspberry Pi
    temp_data = send_command_to_raspberry("temperature", selected_option)

    if "error" in temp_data:
        print(f"Error: {temp_data['error']}")
        exit()

    temperature = temp_data.get("temperature", 0)
    print(json.dumps(temp_data, indent=4))
    print("\nPlease place your finger on the Pulse Oximeter...")
    time.sleep(4)
   

    # Step 2: Request MAX30102 (heart rate & SpO₂) data
    print("\nNow measuring heart rate & SpO₂...")
    max_data = send_command_to_raspberry("max30102")

    if "error" in max_data:
        print(f"Error: {max_data['error']}")
        exit()

    heart_rate = max_data.get("heart_rate", 0)
    spo2 = max_data.get("spo2", 0)
    print(json.dumps(max_data, indent=4))

    # Step 3: Analyze health condition
    print("\n🔍 Analyzing health condition...")
    health_status, health_suggestions = analyze_health(temperature, heart_rate, spo2)

    # Step 4: Display final report
    final_data = {
        "Temperature (deg(C))": temperature,
        "Heart Rate (bpm)": heart_rate,
        "SpO2 (%)": spo2,
        "Health Analysis": health_status,
        "Suggestions": health_suggestions
    }

    print("\n📊 Final Health Report:")
    print(json.dumps(final_data, indent=4))

    # Step 5: Determine overall status
    if "⚠️" in "".join(health_status):
        print("\n🚨 WARNING: You have some health risks. Please follow the suggestions or consult a doctor if needed.")
    else:
        print("\n✅ You are in good health. Keep maintaining a healthy lifestyle! 🎉")


In [2]:
import requests
import runpy

GITHUB_RAW_URL = "https://raw.githubusercontent.com/dark-d07/MEDI-AI/main/bot.py"

response = requests.get(GITHUB_RAW_URL)
if response.status_code == 200:
    script_content = response.text
    exec(script_content)
else:
    print("Failed to fetch the script.")



ChatBot: Hello! Type 'quit' to exit.
User: Hello
ChatBot: Hi there! What can I do for you?
User: Quit
ChatBot: Goodbye!


In [5]:
import json
import random
import sympy as sp
import re
import datetime
import pickle
import numpy as np
import os
import subprocess
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load or create dataset
DATA_FILE = "bot.json"
CHAT_HISTORY_FILE = "chat_history.json"
MODEL_FILE = "chatbot_model.pkl"

if os.path.exists(DATA_FILE):
    with open(DATA_FILE, "r") as file:
        data = json.load(file)
else:
    data = {"responses": {}, "medical_questions": {}, "jokes": [], "fun_facts": [], "medical_facts": []}

responses = data.get("responses", {})
medical_questions = data.get("medical_questions", {})
jokes = data.get("jokes", [])
fun_facts = data.get("fun_facts", [])
medical_facts = data.get("medical_facts", [])

# Combine responses
all_responses = {**responses, **medical_questions}
corpus = list(all_responses.keys())  # User inputs
response_values = list(all_responses.values())  # Bot responses

# Function to save conversation history
def save_chat(user_input, bot_response):
    """Saves chat history."""
    history = []
    if os.path.exists(CHAT_HISTORY_FILE):
        with open(CHAT_HISTORY_FILE, "r") as file:
            history = json.load(file)
    history.append({"user": user_input, "bot": bot_response})
    with open(CHAT_HISTORY_FILE, "w") as file:
        json.dump(history, file, indent=4)

# Function to train and update model
def train_and_save_model():
    """Trains and saves the model."""
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(corpus).toarray()
    with open(MODEL_FILE, "wb") as model_file:
        pickle.dump((vectorizer, vectors, response_values), model_file)

# Load or train model
def load_model():
    """Loads the trained model if it exists."""
    if os.path.exists(MODEL_FILE):
        with open(MODEL_FILE, "rb") as model_file:
            return pickle.load(model_file)
    return None

# Ensure model is up-to-date
train_and_save_model()
model_data = load_model()
vectorizer, vectors, response_values = model_data

# Function to update model with new input
def update_model(new_input, new_response):
    """Adds new input-response pairs and retrains model."""
    if new_input not in all_responses:
        all_responses[new_input] = new_response
        corpus.append(new_input)
        response_values.append(new_response)
        train_and_save_model()  # Retrain the model
        with open(DATA_FILE, "w") as file:
            json.dump(data, file, indent=4)

# Math solver function
def detect_and_solve_math(user_input):
    user_input_cleaned = re.sub(r"(what is|calculate|do|find|solve|compute|work out|answer)", "", user_input, flags=re.IGNORECASE).strip()
    user_input_cleaned = user_input_cleaned.replace("x", "*")
    
    if re.search(r"[\+\-\*/\^]", user_input_cleaned):
        try:
            result = sp.sympify(user_input_cleaned)
            return f"The answer is: {result}"
        except (sp.SympifyError, TypeError):
            return "Sorry, I couldn't solve that math problem."
    
    return None

# Function to check system time
def get_time():
    return "The current time is: " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Function to check health status
def check_health():
    """Runs health_check.py and returns live JSON output."""
    try:
        process = subprocess.Popen(["python3", "health_check.py"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        process.wait()  # Ensure process completes before reading output
        stdout, stderr = process.communicate()  # Read full output

        try:
            data = json.loads(stdout.strip())  # Ensure clean JSON parsing
        except json.JSONDecodeError:
            return "Error: Invalid JSON format received from health_check.py"

        if not data:
            return "Error: No valid health data received."

        return (f"📊 Health Check Data:\n"
                f"🌡️ Ambient Temp: {data['ambient_temp_C']}°C ({data['ambient_temp_F']}°F)\n"
                f"🔥 Forehead Temp: {data['forehead_temp_C']}°C ({data['forehead_temp_F']}°F)")
    except Exception as e:
        return f"Error checking health: {str(e)}"

# Function to get chatbot response
def chatbot_response(user_input):
    user_input = user_input.strip().lower()  # Normalize input

    # Check if it's a math problem
    math_response = detect_and_solve_math(user_input)
    if math_response:
        save_chat(user_input, math_response)
        return math_response
    
    # Check for health queries
    health_keywords = ["check my health", "i have", "i feel", "fever", "heart rate", "temperature"]
    if any(keyword in user_input for keyword in health_keywords):
        health_response = check_health()
        save_chat(user_input, health_response)
        return health_response
    
    # Process text queries
    user_vector = vectorizer.transform([user_input]).toarray()
    similarities = cosine_similarity(user_vector, vectors)
    best_match_index = np.argmax(similarities)

    if similarities[0][best_match_index] > 0.6:  # Higher threshold for better accuracy
        bot_response = response_values[best_match_index]
    else:
        bot_response = "I'm sorry, I don't understand that."

        # Learn from user
        new_response = input("ChatBot: I don't know that yet. How should I respond? ")
        if new_response.strip():
            update_model(user_input, new_response)
            bot_response = "Thank you! I have learned a new response."

    # Special responses
    special_responses = {
        "get_joke": random.choice(jokes) if jokes else "I don't have jokes right now.",
        "get_fact": random.choice(fun_facts) if fun_facts else "I don't have facts right now.",
        "get_medical_fact": random.choice(medical_facts) if medical_facts else "I don't have medical facts right now.",
        "get_time": get_time()
    }
    bot_response = special_responses.get(bot_response, bot_response)

    save_chat(user_input, bot_response)
    return bot_response

# Run chatbot
if __name__ == "__main__":
    print("ChatBot: Hello! Type 'Thank You' to exit.")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "thank you":
            print("ChatBot: Goodbye!")
            break
        print(f"ChatBot: {chatbot_response(user_input)}")


ChatBot: Hello! Type 'Thank You' to exit.
ChatBot: Hi there! What can I do for you?
ChatBot: Thank you! I have learned a new response.
ChatBot: Thank you! I have learned a new response.
ChatBot: Anemia is a condition where the blood lacks enough healthy red blood cells to carry oxygen.
ChatBot: Goodbye!
